## Docker Setup
- Install docker engine for Ubuntu using the convenience script > Post-installation steps.
- Install remote development extension in VSCode.
- Install and configure nvidia container toolkit
- Disconnect VPN
- Don't install Python/Anaconda
- Install correct cuda for latest PyTorch (default cuda may be fine), install default Pytorch
```json
{
  "name": "humble desktop-full",
  "dockerFile": "Dockerfile",
  "runArgs": [
    "--privileged",
    "--network=host",
    "--gpus","all"
  ],
  "workspaceMount": "source=${localWorkspaceFolder},target=/${localWorkspaceFolderBasename},type=bind",
  "workspaceFolder": "/${localWorkspaceFolderBasename}",
  "mounts": [
    "source=${localEnv:HOME}${localEnv:USERPROFILE}/.bash_history,target=/home/vscode/.bash_history,type=bind"
],
"features": {
	"ghcr.io/devcontainers/features/desktop-lite:1": {}
  // "ghcr.io/devcontainers/features/nvidia-cuda:1": {},
  // "ghcr.io/raucha/devcontainer-features/pytorch:1": {}
}
}
```

# ROS2

```bash
gedit ~/.bashrc  
# add: source /opt/ros/humble/setup.bash  
source ~/.bashrc  
```

## Node
```bash
ros2 run PACKAGE NODE  
ros2 run demo_nodes_cpp talker  
ros2 run demo_nodes_cpp listener  
```

Here, /talker node is publishing /chatter topic, and /listener node is subscribing to it.  

ros2 run turtlesim turtlesim_node  

## ROS2 workspace
```bash
sudo apt install python3-colcon-common-extensions  
gedit ~/.bashrc  
add: source /usr/share/colcon_argcomplete/hook/colcon-argcomplete.bash  
source ~/.bashrc  
```

Create a workspace (folder):  
```bash
mkdir ros2_ws && cd ros2_ws && mkdir src  
colcon build  
```

Source the new package installation everytime you want to use the package with:
```bash
source ~/ros2_ws/install/setup.bash  
```
Or,  
```bash
gedit ~/.bashrc  
add: source ~/ros2_ws/install/setup.bash  
source ~/.bashrc  
```
## ROS2 Package
```bash
cd src/  
ros2 pkg create PACKAGE_NAME --build-type ament_python --dependencies rclpy  

cd ..  
colcon build  
```
## ROS2 Node
```bash
cd ros2_ws/ros2_ws/src/my_robot_controller/my_robot_controller$  
touch my_first_node.py  
chmod +x my_first_node.py  
```
My First Node:  
```python
#!/usr/bin/env python3  
import rclpy  
```


## Template node
```python
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node

# We create a Node class and inherit from rclpy.node object
class MyNode(Node):
    # Initiatize the node by proving a node name
    def __init__(self):
        super().__init__("first_node")


def main(args=None):
    # Initiate ros2 communication
    rclpy.init(args=args)

    # Create node
    node = MyNode()

    # Stop ros2 communication
    rclpy.shutdown()

if __name__ == '__main__':
    main ()
```

## Example node: publishes Hello continuously
```python
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node

# We create a Node class and inherit from rclpy.node object
class MyNode(Node):
    # Initiatize the node by proving a node name
    def __init__(self):
        super().__init__("first_node")
        self.counter_ = 0
        self.create_timer(1.0, self.timer_callback)

    def timer_callback(self):
        self.get_logger().info("Hello" + str(self.counter_))
        self.counter_ += 1

def main(args=None):
    # Initiate ros2 communication
    rclpy.init(args=args)

    # Create node
    node = MyNode()

    # Keep the node alive
    rclpy.spin(node)

    # Stop ros2 communication
    rclpy.shutdown()

if __name__ == '__main__':
    main ()
```

To add the node to ros2 run:
src/my_robot_controller/setup.py
```python
    entry_points={
        'console_scripts': [
            # "ExecutableName = PackageName.FileName:main"
            "test_node = my_robot_controller.my_first_node:main"
        ],
```
Then go to workspace folder and execute:
colcon build  --symlink-install
source install/setup.bash [from ros2_ws/ directory]
ros2 run my_robot_controller test_node

ros2 node list
ros2 node info /first_node

## ROS2 Topic
Nodes don't directly talk to each other. Publisher node publishes a Topic, subscriber node can subscribe to it.  
Topics have specified data types, e.g., std_msgs, geometry_msgs, etc.

Print 
ros2 topic list

ros2 topic info /TOPIC_NAME

ros2 topic echo /TOPIC_NAME

ros2 interface show std_msgs/msg/String

ros2 topic hz /TOPIC_NAME for topic publishing rate

## Python Topic Publisher
self.create_publisher(Message_Type, "topic_name", queue_size)

In this case: from geometry_msgs.msg import Twist

In package.xml, we'll add two dependencies:
<depend>geometry_msgs</depend>
<depend>turtlesim</depend>


In [ ]:
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist

class DrawCircleNode(Node):
    def __init__(self) -> None:
        super().__init__("draw_circle")
        self.cmd_vel_pub_ = self.create_publisher(Twist, "/turtle1/cmd_vel", 10)
        self.timer = self.create_timer(0.5, self.send_velocity_command)
        self.get_logger().info("Draw circle node has been started")

    def send_velocity_command(self):
        msg = Twist() # create a msg object from Twist() class
        msg.linear.x = 2.0
        msg.angular.z = 1.0
        self.cmd_vel_pub_.publish(msg)


def main(args=None):
    rclpy.init(args=args)
    node = DrawCircleNode()
    rclpy.spin(node)
    rclpy.shutdown()

## Python Topic Subscriber

self.pose_subscriber = self.create_subscription(topic_type, "topic_name", callback_function, 10)

## Python closed loop control


In [ ]:
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node
from turtlesim.msg import Pose

class poseSubscriberNode(Node):
    def __init__(self):
        super.__init__("pose_subscriber")
        self.pose_subscriber = self.create_subscription(Pose, "turtle1/pose", self.pose_callback, 10)
    
    def pose_callback(self, msg: Pose):
        self.get_logger().info(str(msg))

def main(args=None):
    rclpy.init(args=args)
    node = poseSubscriberNode()
    rclpy.spin(node)
    rclpy.shutdown()

## ROS2 Services
For a request-answer type client-server system.

ros2 run demo_nodes_cpp add_two_ints_server

ros2 service list

ros2 service type /SERVICE_NAME

ros2 interface show /SERVICE_TYPE

ros2 service call /add_two_ints example_interfaces/srv/AddTwoInts "{'a':2,'b':5}"

We use services in two cases:
1. Computation
2. Settings change: e.g., changing pen color of turtlesim.

When to use topic: when we send data.
When to user service: when we need answer to requests.

## ROS2 Setup

```bash
cd my_bot && mkdir src
git clone https://github.com/SunzidHassan/my_bot.git
cd my_bot
# apt-get install python3-pip
- pip install catkin_pkg/apt install catkin_pkg
colcon build --symlink-install
```

#### Building Xacro robot description
Everytime we add a new file:  
```bash
colcon build --symlink-install
```

#### State publisher
Then we need to source the updated install in state publisher, gazebo, etc.:  
```bash
source install/setup.bash
```

State publisher: 
```bash
ros2 launch blue_bot rsp.launch.py use_sim_time:=true
```

#### Install Gazebo
```bash
# dependencies sudo apt install __
sudo apt install ros-${ROS_DISTRO}-twist-mux
sudo apt install ros-${ROS_DISTRO}-twist-stamper
sudo apt install ros-${ROS_DISTRO}-ros2-control
sudo apt install ros-${ROS_DISTRO}-ros2-controllers

# ros2 control
sudo apt install ros-${ROS_DISTRO}-gz-ros2-control

sudo apt update && sudo apt upgrade -y
sudo apt-get install ros-${ROS_DISTRO}-ros-gz
# sudo apt install ros-humble-gazebo-ros-pkgs
```

#### Gazebo simulation
```bash
rosdep update
```

Gazebo launch:  
```bash
ros2 launch ros_gz_sim gz_sim.launch.py # Jazzy
ros2 launch gazebo_ros gazebo.launch.py # Humble
```

Launching with launcher file:  
```bash
ros2 launch blue_bot launch_sim.launch.py
```

If the normal or 3d camera is not working, try
- colcon build and source setup.bash.
- starting from a fresh gazebo env,
- install python

Launching with launcher in custom simulation environment:  
```bash
ros2 launch my_bot launch_sim.launch.py world:=./src/my_bot/worlds/football.world 
```

#### Rviz load view
```bash
rviz2 -d src/my_bot/config/view_bot.rviz
```
#### Teleop
```bash
ros2 run teleop_twist_keyboard teleop_twist_keyboard
```

### Gazebo

ROS uses URDF file for models, Gazebo uses SDF files. An entire environment is an SDF file, and each element in that world can be an SDF file.

## ROS2 Jazzy

Cleaning build artifacts (deleted packages):

cd ~/ros2_ws
rm -rf build/ install/ log/

## 